In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
y_lookup = dict(
    pd.read_csv(
        "datasets/task_1_depression/risk-golden-truth-test.txt",
        sep="\t",
        header=None,
    ).to_records(index=False)
)


In [ ]:
df = pd.read_csv("datasets/task_1_depression/depression_merged.csv")

In [ ]:
def to_liwc_token(text):
    for match in re.finditer(r'\w+', text, re.UNICODE):
        yield match.group(0).lower()

In [ ]:
import liwc
liwc_parse, categories = liwc.load_token_parser('dic/LIWC2007_English080730.dic')
K = len(categories)
kat_lookup = dict(zip(categories, range(K)))

In [ ]:
y_lookup = dict(
    pd.read_csv(
        "datasets/task_1_depression/risk-golden-truth-test.txt",
        sep="\t",
        header=None,
    ).to_records(index=False)
)


In [ ]:
subjects = sorted(df.user.drop_duplicates().to_list())
S = len(subjects)
subj_lookup =  dict(zip(subjects, range(S)))
subj_ts = dict()

In [ ]:
# generate time serie
for subj, dfi in tqdm(df.groupby("user")):
    subj_ts[subj] = list()

    for n, text in enumerate(dfi.sort_values("date_time").text):
        kat_freq = np.zeros(K)
        for t in to_liwc_token(text):
            for m in liwc_parse(t):
                k = kat_lookup[m]
                kat_freq[k] += 1

        if not kat_freq.sum():
            continue

        kat_freq /= kat_freq.sum()
        subj_ts[subj].append(kat_freq)
    subj_ts[subj] = np.array(subj_ts[subj])


In [ ]:
MIN_LENGTH = 20
subj_ts = {k:v for k, v in subj_ts.items() if not v.shape[0] < MIN_LENGTH}
subj_sign = { k:np.sign(v) for k,v in subj_ts.items()}
y_lookup = {k:v  for k,v in y_lookup.items() if k in subj_ts}

"""
    classificatore sui amplificatoti nella trasformata di fourier (fft fast fourier)
"""

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from  statsmodels.tsa.arima.model import ARIMA

In [ ]:
from erisk import erde_evaluation

def score(out_file):
    erde_evaluation("datasets/task_1_depression/risk-golden-truth-test.txt", out_file)

In [ ]:
k = categories.index("family")
s = "subject3414"


In [ ]:
def ma(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w

In [ ]:
ts = subj_sign[s][:50,k]
ts = ma(ts, 3)
ts = subj_ts[s][:50,k]

In [ ]:
model = ARIMA(ts, order=(1,3,2))
results = model.fit()
plt.plot(ts)
plt.plot(results.fittedvalues[1:], color='red', )

bazuka

In [ ]:
X, y = np.zeros((len(y_lookup), 64, 2000)), []
for i, subj in enumerate(y_lookup):
    y.append(y_lookup[subj])
    X_unpad = subj_ts[subj]

    ts_len = X_unpad.T.shape[-1]
    X[i, :, :ts_len] = X_unpad.T

y = np.array(y)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

In [ ]:
aaa_path = r"MLSTM_FCN/data/aaa/"
np.save(aaa_path + 'X_train.npy', X_train)
np.save(aaa_path + 'y_train.npy', y_train)
np.save(aaa_path + 'X_test.npy', X_test)
np.save(aaa_path + 'y_test.npy', y_test)

In [ ]:
from MLSTM_FCN import aaa_model